<a href="https://www.kaggle.com/code/harshyadav02/lgbm-for-both-defog-tdcsfog?scriptVersionId=154979453" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# import library
import os
import random
import cv2
import pandas as pd
import numpy as np

## Pre-processing (DEFOG)🏡

In [2]:
#reference: https://www.kaggle.com/code/ghrangel/read-data-and-merge

DATA_ROOT_DEFOG = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog/'
defog = pd.DataFrame()
for root, dirs, files in os.walk(DATA_ROOT_DEFOG):
    for name in files:       
        f = os.path.join(root, name)
        df_list= pd.read_csv(f)
        df_list['file']= name.split('.')[0]
        defog = pd.concat([defog, df_list], axis=0)

defog

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,file
0,0,-1.002697,0.022371,0.068304,0,0,0,False,False,be9d33541d
1,1,-1.002641,0.019173,0.066162,0,0,0,False,False,be9d33541d
2,2,-0.999820,0.019142,0.067536,0,0,0,False,False,be9d33541d
3,3,-0.998023,0.018378,0.068409,0,0,0,False,False,be9d33541d
4,4,-0.998359,0.016726,0.066448,0,0,0,False,False,be9d33541d
...,...,...,...,...,...,...,...,...,...,...
109120,109120,-0.939241,0.031564,-0.394737,0,0,0,False,False,06414383cf
109121,109121,-0.941096,0.031582,-0.392626,0,0,0,False,False,06414383cf
109122,109122,-0.940131,0.029092,-0.394385,0,0,0,False,False,06414383cf
109123,109123,-0.939872,0.028058,-0.398664,0,0,0,False,False,06414383cf


In [3]:
defog['file'].unique().shape

(91,)

In [4]:
defog = defog[(defog['Task']==1)&(defog['Valid']==1)]

In [5]:
print('the shape of defog dataset is {}'.format(defog.shape))

the shape of defog dataset is (4090530, 10)


##  Combining it with 🏡defog-metadata.

In [6]:
defog_metadata = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv")
defog_metadata

,Id,Subject,Visit,Medication
0,02ab235146,e1f62e,2,on
1,02ea782681,ae2d35,2,on
2,06414383cf,8c1f5e,2,off
3,092b4c1819,2874c5,1,off
4,0a900ed8a2,0e3d49,2,on
...,...,...,...,...
132,f3a921edee,1a778d,1,off
133,f40e8c6ebe,575c60,1,off
134,f8ddbdd98d,107712,1,on
135,f9efef91fb,5d9cae,2,off


In [7]:
defog_metadata['Subject'].unique().shape

(45,)

In [8]:
defog_m= defog_metadata.merge(defog, how = 'inner', left_on = 'Id', right_on = 'file')
defog_m.drop(['file','Valid','Task'], axis = 1, inplace = True)
defog_m

,Id,Subject,Visit,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,02ea782681,ae2d35,2,on,1000,-0.970018,0.061626,-0.265625,0,0,0
1,02ea782681,ae2d35,2,on,1001,-0.984375,0.044497,-0.265625,0,0,0
2,02ea782681,ae2d35,2,on,1002,-0.984375,0.029016,-0.265625,0,0,0
3,02ea782681,ae2d35,2,on,1003,-0.984375,0.015625,-0.265625,0,0,0
4,02ea782681,ae2d35,2,on,1004,-0.984670,0.015330,-0.265625,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4090525,f9fc61ce85,040587,1,on,119027,-0.961216,0.142428,-0.289655,0,0,0
4090526,f9fc61ce85,040587,1,on,119028,-0.960343,0.142836,-0.290506,0,0,0
4090527,f9fc61ce85,040587,1,on,119029,-0.957958,0.145494,-0.290007,0,0,0
4090528,f9fc61ce85,040587,1,on,119030,-0.960616,0.145839,-0.291527,0,0,0


In [9]:
# Create an empty DataFrame to store the sampled data
sampled_df = pd.DataFrame(columns=defog_m.columns)

# Loop through each condition
for condition_column in ['StartHesitation', 'Turn', 'Walking']:
    # Sample 500 records for each condition
    condition_samples = defog_m[defog_m[condition_column] == 1].sample(n=500, random_state=42)
    sampled_df = pd.concat([sampled_df, condition_samples], ignore_index=True)

# Sample 500 records for the DefaultClass condition (all three values are 0)
default_class_samples = defog_m[(defog_m['StartHesitation'] == 0) & (defog_m['Turn'] == 0) &
                                (defog_m['Walking'] == 0)].sample(n=500, random_state=42)
sampled_df = pd.concat([sampled_df, default_class_samples], ignore_index=True)

# Now 'sampled_df' contains 500 records for each condition and 500 records for the DefaultClass



In [10]:
sampled_df

,Id,Subject,Visit,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,bf2fd0ff35,d89567,1,off,36382,-0.796520,0.060129,-0.490544,1,0,0
1,0d7ab3a9f9,8c1f5e,2,on,18223,-0.933665,-0.133591,-0.400478,1,0,0
2,bf2fd0ff35,d89567,1,off,36395,-0.807429,-0.263876,-0.497874,1,0,0
3,68e7e02a47,08de77,1,off,112495,-0.902516,-0.031580,-0.282323,1,0,0
4,1d99c2eecf,12f8d1,1,on,141107,-0.923095,0.078125,-0.484375,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1995,b3891f42d0,3e1d75,1,off,42856,-1.015625,0.078125,-0.031250,0,0,0
1996,a2f1a8ab76,e86b6e,2,on,34741,-0.984986,0.000673,-0.059040,0,0,0
1997,4520cf1068,3e1d75,1,on,82017,-0.968750,0.038287,0.055846,0,0,0
1998,02ea782681,ae2d35,2,on,30416,-1.000000,0.078125,-0.218750,0,0,0


In [11]:
# summary table function
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values * 100
    summ['%missing'] = df.isnull().sum().values / len(df)
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [12]:
summary(sampled_df)

data shape: (2000, 11)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
Id,object,0,0.0,89,NaN,NaN,bf2fd0ff35,0d7ab3a9f9,bf2fd0ff35
Subject,object,0,0.0,38,NaN,NaN,d89567,8c1f5e,d89567
Visit,object,0,0.0,2,NaN,NaN,1,2,1
Medication,object,0,0.0,2,NaN,NaN,off,on,off
Time,object,0,0.0,1992,NaN,NaN,36382,18223,36395
AccV,float64,0,0.0,1856,-1.552976,-0.359375,-0.79652,-0.933665,-0.807429
AccML,float64,0,0.0,1860,-0.481112,0.502826,0.060129,-0.133591,-0.263876
AccAP,float64,0,0.0,1851,-1.0,0.638534,-0.490544,-0.400478,-0.497874
StartHesitation,object,0,0.0,2,NaN,NaN,1,1,1
Turn,object,0,0.0,2,NaN,NaN,0,0,0


In [13]:
sampled_df['StartHesitation'].value_counts()

0    1500
1     500
Name: StartHesitation, dtype: int64

In [14]:
sampled_df['Turn'].value_counts()

0    1500
1     500
Name: Turn, dtype: int64

In [15]:
sampled_df['Walking'].value_counts()

0    1500
1     500
Name: Walking, dtype: int64

In [16]:
# garbage collection for memory
import gc
gc.collect()

84

## Pre-Processing (TDCSFOG)🥼

In [17]:
DATA_ROOT_TDCSFOG = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/'
tdcsfog = pd.DataFrame()
for root, dirs, files in os.walk(DATA_ROOT_TDCSFOG):
    for name in files:       
        f = os.path.join(root, name)
        df_list= pd.read_csv(f)
        words = name.split('.')[0]
        df_list['file']= name.split('.')[0]
        tdcsfog = pd.concat([tdcsfog, df_list], axis=0)
tdcsfog

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,file
0,0,-9.665890,0.042550,0.184744,0,0,0,a171e61840
1,1,-9.672969,0.049217,0.184644,0,0,0,a171e61840
2,2,-9.670260,0.033620,0.193790,0,0,0,a171e61840
3,3,-9.673356,0.035159,0.184369,0,0,0,a171e61840
4,4,-9.671458,0.043913,0.197814,0,0,0,a171e61840
...,...,...,...,...,...,...,...,...
5153,5153,-9.915920,-0.105897,-1.123455,0,0,0,0506d9a39f
5154,5154,-9.693752,-0.066892,-1.114903,0,0,0,0506d9a39f
5155,5155,-9.548118,-0.098315,-1.112123,0,0,0,0506d9a39f
5156,5156,-9.469803,-0.111004,-1.130814,0,0,0,0506d9a39f


##  Combining it with 🥼tdcsfog-metadata.

In [18]:
tdcsfog_metadata = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv")
tdcsfog_metadata

,Id,Subject,Visit,Test,Medication
0,003f117e14,4dc2f8,3,2,on
1,009ee11563,f62eec,4,2,on
2,011322847a,231c3b,2,2,on
3,01d0fe7266,231c3b,2,1,off
4,024418ba39,fa8764,19,3,on
...,...,...,...,...,...
828,feba449e1a,312788,19,1,on
829,ff4f844fd3,af82b2,2,3,on
830,ff53514514,3b2403,2,3,on
831,ff92d9244d,2d57c2,20,2,on


In [19]:
tdcsfog_m= tdcsfog_metadata.merge(tdcsfog, how = 'inner', left_on = 'Id', right_on = 'file')
tdcsfog_m.drop(['file'], axis = 1, inplace = True)
tdcsfog_m

,Id,Subject,Visit,Test,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,003f117e14,4dc2f8,3,2,on,0,-9.533939,0.566322,-1.413525,0,0,0
1,003f117e14,4dc2f8,3,2,on,1,-9.536140,0.564137,-1.440621,0,0,0
2,003f117e14,4dc2f8,3,2,on,2,-9.529345,0.561765,-1.429332,0,0,0
3,003f117e14,4dc2f8,3,2,on,3,-9.531239,0.564227,-1.415490,0,0,0
4,003f117e14,4dc2f8,3,2,on,4,-9.540825,0.561854,-1.429471,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7062667,ffda8fadfd,7fcee9,20,1,off,4220,-9.403467,0.089003,-3.220304,0,0,0
7062668,ffda8fadfd,7fcee9,20,1,off,4221,-9.404246,0.090531,-3.216584,0,0,0
7062669,ffda8fadfd,7fcee9,20,1,off,4222,-9.405770,0.084380,-3.224039,0,0,0
7062670,ffda8fadfd,7fcee9,20,1,off,4223,-9.403579,0.084236,-3.236686,0,0,0


In [20]:
# Create an empty DataFrame to store the sampled data
tdcs_sampled_df = pd.DataFrame(columns=defog_m.columns)

# Loop through each condition
for condition_column in ['StartHesitation', 'Turn', 'Walking']:
    # Sample 500 records for each condition
    condition_samples = tdcsfog_m[tdcsfog_m[condition_column] == 1].sample(n=207838, random_state=42)
    tdcs_sampled_df = pd.concat([tdcs_sampled_df, condition_samples], ignore_index=True)

# Sample 500 records for the DefaultClass condition (all three values are 0)
default_class_samples = tdcsfog_m[(tdcsfog_m['StartHesitation'] == 0) & (tdcsfog_m['Turn'] == 0) &
                                (tdcsfog_m['Walking'] == 0)].sample(n=207838, random_state=42)
tdcs_sampled_df = pd.concat([tdcs_sampled_df, default_class_samples], ignore_index=True)

# Now 'sampled_df' contains 500 records for each condition and 500 records for the DefaultClass



In [21]:
summary(tdcs_sampled_df)

data shape: (831352, 12)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
Id,object,0,0.0,833,NaN,NaN,e318182fe2,2d4c5f6c17,be8fdfa712
Subject,object,0,0.0,62,NaN,NaN,79011a,b19f77,194d1d
Visit,object,0,0.0,7,NaN,NaN,3,2,2
Medication,object,0,0.0,2,NaN,NaN,on,on,off
Time,object,0,0.0,88318,NaN,NaN,575,2095,49407
AccV,float64,0,0.0,826336,-26.477008,5.797328,-9.860968,-9.896153,-10.027954
AccML,float64,0,0.0,827434,-23.241254,24.540494,0.71676,0.550487,-0.902985
AccAP,float64,0,0.0,826852,-30.231809,29.341835,1.636743,1.023587,0.703142
StartHesitation,object,0,0.0,2,NaN,NaN,1,1,1
Turn,object,0,0.0,2,NaN,NaN,0,0,0


In [22]:
# garbage collection for memory
import gc
gc.collect()

21

# Feature engineering and modeling (DEFOG)🏡

In [23]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [24]:
conditions = [
    (sampled_df['StartHesitation'] == 1),
    (sampled_df['Turn'] == 1),
    (sampled_df['Walking'] == 1)]
choices = ['StartHesitation', 'Turn', 'Walking']
sampled_df['event'] = np.select(conditions, choices, default='Normal')

In [25]:
sampled_df['event'].value_counts().to_frame().style.background_gradient()

,event
StartHesitation,500
Turn,500
Walking,500
Normal,500


In [26]:
train_df = sampled_df[['AccV','AccML','AccAP','event']]

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_df['target'] = le.fit_transform(train_df['event'])

In [28]:
X = train_df.drop(['event','target'], axis=1)
y = train_df['target']

In [29]:
X

,AccV,AccML,AccAP
0,-0.796520,0.060129,-0.490544
1,-0.933665,-0.133591,-0.400478
2,-0.807429,-0.263876,-0.497874
3,-0.902516,-0.031580,-0.282323
4,-0.923095,0.078125,-0.484375
...,...,...,...
1995,-1.015625,0.078125,-0.031250
1996,-0.984986,0.000673,-0.059040
1997,-0.968750,0.038287,0.055846
1998,-1.000000,0.078125,-0.218750


In [30]:
y

0       1
1       1
2       1
3       1
4       1
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Name: target, Length: 2000, dtype: int64

In [31]:
y.unique()

array([1, 2, 3, 0])

In [32]:
y.value_counts()

1    500
2    500
3    500
0    500
Name: target, dtype: int64

In [33]:
# spliting dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1004)

In [34]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [35]:
# Make predictions on the test set
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Import the accuracy_score function

y_pred = rf_classifier.predict(X_test)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.5733333333333334
Confusion Matrix:
[[ 59  17  40  35]
 [  6 119  12   9]
 [ 22  21  83  20]
 [ 34  20  20  83]]
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.39      0.43       151
           1       0.67      0.82      0.74       146
           2       0.54      0.57      0.55       146
           3       0.56      0.53      0.55       157

    accuracy                           0.57       600
   macro avg       0.57      0.58      0.57       600
weighted avg       0.56      0.57      0.57       600



In [36]:
print(y_pred[0:999])

[2 1 1 1 1 2 3 2 1 1 2 3 0 0 1 2 2 1 2 2 3 0 0 2 1 0 2 1 1 2 1 0 1 1 0 1 2
 2 0 3 2 0 0 2 3 0 3 1 3 1 2 2 0 1 3 2 3 1 1 2 2 1 2 1 2 3 1 3 1 1 2 3 2 0
 2 1 3 0 0 0 0 1 3 3 1 1 0 0 1 0 2 3 1 1 1 1 3 2 2 2 0 2 3 0 1 3 0 3 2 0 2
 2 2 1 0 0 3 3 2 3 1 3 2 0 1 2 0 0 3 2 0 2 2 3 0 0 1 2 3 3 0 0 1 2 1 1 3 0
 0 1 2 0 0 3 0 1 1 1 2 0 2 3 1 1 1 2 0 3 2 3 1 3 3 2 1 2 1 0 1 1 3 1 1 2 3
 3 2 1 0 2 0 2 2 1 0 1 2 0 0 3 1 1 2 0 0 2 1 1 1 1 3 0 2 3 3 3 1 2 3 3 0 3
 2 2 2 3 2 3 2 1 3 3 0 1 2 2 2 2 3 3 3 3 3 2 2 1 3 1 2 3 1 1 3 2 2 3 1 1 2
 3 0 0 0 3 0 2 1 3 2 3 1 2 3 3 3 1 0 1 2 0 2 2 2 3 2 0 3 0 2 1 3 3 3 2 1 2
 2 2 3 0 1 1 0 2 3 3 2 3 1 2 1 0 1 0 2 2 0 0 0 1 1 1 1 3 0 3 2 2 0 1 3 0 1
 3 0 1 0 2 0 2 2 2 2 0 1 1 0 1 3 0 1 0 3 1 3 1 1 0 2 2 3 2 0 3 3 0 0 3 1 3
 2 1 1 3 3 3 2 3 3 2 1 0 3 3 3 1 0 2 2 0 1 1 3 0 2 1 2 0 1 1 1 1 0 2 1 2 1
 2 2 1 2 3 1 2 2 1 3 1 1 3 1 2 2 2 3 1 1 2 1 1 1 1 1 1 0 2 0 3 1 1 2 3 3 1
 1 0 3 3 0 2 3 0 3 3 3 1 0 3 2 2 2 2 0 0 3 2 0 0 3 1 0 3 2 0 2 1 1 2 1 1 1
 0 3 1 2 0 1 1 2 2 3 1 3 

In [37]:
test_defog_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/02ab235146.csv'
test_defog = pd.read_csv(test_defog_path)
name = os.path.basename(test_defog_path)
id_value = name.split('.')[0]
test_defog['Id_value'] = id_value
test_defog['Id'] = test_defog['Id_value'].astype(str) + '_' + test_defog['Time'].astype(str)
test_defog = test_defog[['Id','AccV','AccML','AccAP']]
test_defog.set_index('Id',inplace=True)

In [38]:
# predict event probability
test_defog_pred=rf_classifier.predict(test_defog)
test_defog['event'] = test_defog_pred

In [39]:
test_defog

,AccV,AccML,AccAP,event
Id,,,,
02ab235146_0,-0.914652,-0.300851,0.298156,0
02ab235146_1,-0.914367,-0.301572,0.298325,0
02ab235146_2,-0.779540,-0.343197,0.275690,0
02ab235146_3,-0.993162,-0.270281,0.315775,0
02ab235146_4,-0.948767,-0.307849,0.297191,0
...,...,...,...,...
02ab235146_281683,-0.899299,-0.232668,0.389147,0
02ab235146_281684,-0.901973,-0.233051,0.390114,0
02ab235146_281685,-0.901690,-0.231888,0.391896,0


In [40]:
# expand event column it to three columns
test_defog['StartHesitation'] = np.where(test_defog['event']==1, 1, 0)
test_defog['Turn'] = np.where(test_defog['event']==2, 1, 0)
test_defog['Walking'] = np.where(test_defog['event']==3, 1, 0)

In [41]:
test_defog.shape

(281688, 7)

In [42]:
print(test_defog['event'].value_counts(),'\n')
print(test_defog['StartHesitation'].value_counts(),'\n')
print(test_defog['Turn'].value_counts(),'\n')
print(test_defog['Walking'].value_counts(),'\n')

0    156359
3     75020
2     39948
1     10361
Name: event, dtype: int64 

0    271327
1     10361
Name: StartHesitation, dtype: int64 

0    241740
1     39948
Name: Turn, dtype: int64 

0    206668
1     75020
Name: Walking, dtype: int64 



# Feature engineering and modeling (TDCSFOG)🥼

In [43]:
conditions = [
    (tdcs_sampled_df['StartHesitation'] == 1),
    (tdcs_sampled_df['Turn'] == 1),
    (tdcs_sampled_df['Walking'] == 1)]
choices = ['StartHesitation', 'Turn', 'Walking']
tdcs_sampled_df['event'] = np.select(conditions, choices, default='Normal')

In [44]:
tdcs_sampled_df['event'].value_counts().to_frame().style.background_gradient()

,event
StartHesitation,207838
Turn,207838
Walking,207838
Normal,207838


In [45]:
train_df = tdcs_sampled_df[['AccV','AccML','AccAP','event']]

In [46]:
train_df['target'] = le.fit_transform(train_df['event'])

In [47]:
X = train_df.drop(['event','target'], axis=1)
y = train_df['target']

In [48]:
# split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1004)


In [49]:
# LGBM
from lightgbm import LGBMClassifier

# Initialize the LightGBM classifier
lgbm_classifier = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the classifier
lgbm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_2 = lgbm_classifier.predict(X_test)

In [50]:
# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred_2)
conf_matrix = confusion_matrix(y_test, y_pred_2)
classification_rep = classification_report(y_test, y_pred_2)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.463833139874061
Confusion Matrix:
[[24425  6427  7997  2331]
 [ 3581 19358  8500  9968]
 [ 8901  9119 16755  6878]
 [ 5348 12356  7743 16584]]
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.59      0.59     41180
           1       0.41      0.47      0.44     41407
           2       0.41      0.40      0.41     41653
           3       0.46      0.39      0.43     42031

    accuracy                           0.46    166271
   macro avg       0.47      0.46      0.46    166271
weighted avg       0.46      0.46      0.46    166271



In [51]:
test_tdcsfog_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/003f117e14.csv'
test_tdcsfog = pd.read_csv(test_tdcsfog_path)
name = os.path.basename(test_tdcsfog_path)
id_value = name.split('.')[0]
test_tdcsfog['Id_value'] = id_value
test_tdcsfog['Id'] = test_tdcsfog['Id_value'].astype(str) + '_' + test_tdcsfog['Time'].astype(str)
test_tdcsfog = test_tdcsfog[['Id','AccV','AccML','AccAP']]
test_tdcsfog.set_index('Id',inplace=True)

In [52]:
test_tdcsfog

,AccV,AccML,AccAP
Id,,,
003f117e14_0,-9.533939,0.566322,-1.413525
003f117e14_1,-9.536140,0.564137,-1.440621
003f117e14_2,-9.529345,0.561765,-1.429332
003f117e14_3,-9.531239,0.564227,-1.415490
003f117e14_4,-9.540825,0.561854,-1.429471
...,...,...,...
003f117e14_4677,-9.351431,0.370047,-2.286318
003f117e14_4678,-9.341410,0.347721,-2.321991
003f117e14_4679,-9.350667,0.361341,-2.297158


In [53]:
test_tdcsfog_pred=lgbm_classifier.predict(test_tdcsfog)
test_tdcsfog['event'] = test_tdcsfog_pred

In [54]:
test_tdcsfog

,AccV,AccML,AccAP,event
Id,,,,
003f117e14_0,-9.533939,0.566322,-1.413525,0
003f117e14_1,-9.536140,0.564137,-1.440621,0
003f117e14_2,-9.529345,0.561765,-1.429332,0
003f117e14_3,-9.531239,0.564227,-1.415490,0
003f117e14_4,-9.540825,0.561854,-1.429471,0
...,...,...,...,...
003f117e14_4677,-9.351431,0.370047,-2.286318,0
003f117e14_4678,-9.341410,0.347721,-2.321991,0
003f117e14_4679,-9.350667,0.361341,-2.297158,0


In [55]:
test_tdcsfog['StartHesitation'] = np.where(test_tdcsfog['event']==1, 1, 0)
test_tdcsfog['Turn'] = np.where(test_tdcsfog['event']==2, 1, 0)
test_tdcsfog['Walking'] = np.where(test_tdcsfog['event']==3, 1, 0)
test_tdcsfog.reset_index('Id', inplace=True)

In [56]:
test_tdcsfog

,Id,AccV,AccML,AccAP,event,StartHesitation,Turn,Walking
0,003f117e14_0,-9.533939,0.566322,-1.413525,0,0,0,0
1,003f117e14_1,-9.536140,0.564137,-1.440621,0,0,0,0
2,003f117e14_2,-9.529345,0.561765,-1.429332,0,0,0,0
3,003f117e14_3,-9.531239,0.564227,-1.415490,0,0,0,0
4,003f117e14_4,-9.540825,0.561854,-1.429471,0,0,0,0
...,...,...,...,...,...,...,...,...
4677,003f117e14_4677,-9.351431,0.370047,-2.286318,0,0,0,0
4678,003f117e14_4678,-9.341410,0.347721,-2.321991,0,0,0,0
4679,003f117e14_4679,-9.350667,0.361341,-2.297158,0,0,0,0
4680,003f117e14_4680,-9.343977,0.365726,-2.292650,0,0,0,0


In [57]:
test_tdcsfog.shape

(4682, 8)

In [58]:
print(test_tdcsfog['event'].value_counts(),'\n')
print(test_tdcsfog['StartHesitation'].value_counts(),'\n')
print(test_tdcsfog['Turn'].value_counts(),'\n')
print(test_tdcsfog['Walking'].value_counts(),'\n')

0    2043
1    1452
2     979
3     208
Name: event, dtype: int64 

0    3230
1    1452
Name: StartHesitation, dtype: int64 

0    3703
1     979
Name: Turn, dtype: int64 

0    4474
1     208
Name: Walking, dtype: int64 



In [59]:
submit = pd.concat([test_tdcsfog,test_defog])
submit = submit[['Id', 'StartHesitation', 'Turn','Walking']]

In [60]:
submit.to_csv('/kaggle/working/submission.csv', index=False)

In [61]:
sample = pd.read_csv('/kaggle/working/submission.csv')